In [2]:
from google.colab import drive
drive.mount('/content/drive')

# https://keras.io/api/layers/recurrent_layers/lstm/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Practical Materials - Lab 5/Practical Materials - Lab 5

/content/drive/MyDrive/Practical Materials - Lab 5/Practical Materials - Lab 5


In [4]:
!pip install matplotlib-venn
!pip install np_utils

In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# from tensorflow.keras.utils import np_utils
# from keras.utils.np_utils import to_categorical
utils.to_categorical

<function keras.src.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

### **Load the ascii text and conver to lowercase**

In [6]:
filename = "data.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

## **Load the 1st 100 words**
Indexing start with 0

In [7]:
#we can change this according to our priority
raw_text[0:100]

"project gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of"

## **TO DO :**
preprocess the data set and remove unwanted characters

## **Creating the caharacters in to numerical characters, we need to use the dictionary**

In [8]:
chars = sorted(list(set(raw_text)))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
58


In [9]:
char_to_int = dict((c,i) for i,c in enumerate(chars))
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57}


## **Claculate how many characters and words in this text file**

In [10]:
n_chars = len(raw_text)
n_vocab = len(chars)

print("Total Characters : ", n_chars)

print("Total Vocab : Unique characters : ", n_vocab)

Total Characters :  163780
Total Vocab : Unique characters :  58


In this exercise we will split the book text up into subsequences with a fixed length of 100 characters..
after going 100 characters we are going to predict next character..

ex :- CHAPTER (SEQ LEN = 5)
CHAPT -> E HAPTE -> R

In [11]:
#TO DO
#Change the value of sequence length and check the training dataset
#Can also add test dataset

In [12]:
#Prepare the dataset of imput to output pairs encoded as integers

seq_length = 100  #can changed
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataY)
print("Total Patterns : ", n_patterns)


Total Patterns :  163680


In [13]:
print(dataX[i])

[1, 39, 36, 43, 47, 1, 47, 49, 46, 35, 52, 34, 36, 1, 46, 52, 49, 1, 45, 36, 54, 1, 36, 33, 46, 46, 42, 50, 11, 1, 32, 45, 35, 1, 39, 46, 54, 1, 51, 46, 0, 50, 52, 33, 50, 34, 49, 40, 33, 36, 1, 51, 46, 1, 46, 52, 49, 1, 36, 44, 32, 40, 43, 1, 45, 36, 54, 50, 43, 36, 51, 51, 36, 49, 1, 51, 46, 1, 39, 36, 32, 49, 1, 32, 33, 46, 52, 51, 1, 45, 36, 54, 1, 36, 33, 46, 46, 42, 50, 13]


In [14]:
print(dataY[i])

0


Transform the list of input sequences into the form [samples, time steps, features] that is expected by an
LSTM network and rescale the integers to the range [0,1] to make the patterns easier to learn by the LSTM
network that uses the sigmoid activation function by default.

In [15]:
#one hot encoding
#taking y values from 0 to 1

#resharp X to be [samples, time steps, features]
X = numpy.reshape(dataX,(n_patterns, seq_length,1))

#normalize-rescaling the integer values
X = X / float(n_vocab)
print('X : \n', X)

X : 
 [[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.01724138]
  [0.79310345]
  [0.63793103]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.79310345]
  [0.63793103]
  [0.01724138]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.63793103]
  [0.01724138]
  [0.55172414]]

 ...

 [[0.87931034]
  [0.79310345]
  [0.01724138]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.79310345]
  [0.01724138]
  [0.67241379]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.01724138]
  [0.67241379]
  [0.62068966]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


In [16]:
#Convert the output values (single characters converted to integers) into a one hot encoding.
y = utils.to_categorical(dataY)
print(y)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [17]:
#TO DO
#Change the configuration of the model
#(adding more layers, change LSTM units, change Dropout values, optimizor function)

In [18]:
#define the LSTM model

model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1], X.shape[2]))) #can have 1 or more training samples
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam")

#define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
#you can remove "-{epoch:02d}-{loss:.4f}.hdf5" AND GIVE "weights-improvement" ONLY
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min'),
callbacks_list = [checkpoint]

#checkpoint - 1st epoch lower validation accuracy
              # 2nd higher validation accuracy
              # 3rd lower validation accuracy

In [19]:
#Fitting the model to data
epochs = 3
batch_size = 128

In [20]:
model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

Epoch 1/3
1279/1279 [==============================] - ETA: 0s - loss: 2.9943
Epoch 1: loss improved from inf to 2.99433, saving model to weights-improvement-01-2.9943.hdf5
1279/1279 [==============================] - 868s 676ms/step - loss: 2.9943
Epoch 2/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1279/1279 [==============================] - ETA: 0s - loss: 2.8053
Epoch 2: loss improved from 2.99433 to 2.80530, saving model to weights-improvement-02-2.8053.hdf5
1279/1279 [==============================] - 836s 654ms/step - loss: 2.8053
Epoch 3/3
1279/1279 [==============================] - ETA: 0s - loss: 2.7291
Epoch 3: loss improved from 2.80530 to 2.72908, saving model to weights-improvement-03-2.7291.hdf5
1279/1279 [==============================] - 830s 649ms/step - loss: 2.7291


## **Greating text with LSTM Model**

In [26]:
#load the network weights
filename = "weights-improvement-03-2.7291.hdf5"
# 2.3100 IS MINIMUM LOSS and we have to enter our minimum loss here
# 02 is number of epochs
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [28]:
#Gerenate Random seed
#generate random set of characters and find what is the next 100 words

print(len(dataX))

start = numpy.random.randint(0, len(dataX)-1)
print(start)
pattern = dataX[start] #dataX contains list of patters
print("Seed : ")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

163680
43822
Seed : 
" mong
the trees, a little sharp bark just over her head made her look up in a
great hurry.

an enormo "


In [29]:
#Generate Characters
length = 10
final = []

for i in range(length):

  #reshaping the seed sequence before passing it into LSTM model
  x = numpy.reshape(pattern, (1,len(pattern),1))
  # print(x)

  #normalizing the integer values
  x = x/float(n_vocab)
  # print(x)

  #making prediction
  prediction = model.predict(x, verbose=0)

  #get the predicted value with maximum probability
  index = numpy.argmax(prediction)

  #Convert the prediction integer to char
  result = int_to_char[index]
  print(result)
  final.append(result)

  #Adding the predicted characters to the sequence
  pattern.append(index)

  #Removing the first character from the seed sequence
  pattern = pattern[1:len(pattern)]

print(final)

 
t
o
e
 
t
o
e
t
 
[' ', 't', 'o', 'e', ' ', 't', 'o', 'e', 't', ' ']


In [31]:
# %tensorflow_version 2.X

# import tensorflow as tf

# device_name = tf.test.gpu_device_name()

# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at : {}'.format(device_name))